Import the necessary modules

In [ ]:
from pathlib import Path
import geopandas
import pandas
import sys
import os
from pathlib import Path

pandas.set_option("display.max_columns", None)

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path+str(Path("/")))
import geoutilities.convert as convert
import geoutilities.slopeAspect as slopeAspect

First, load a parcels file by entering its location into filePath

    Example:
                filePath = Path("../data/sampleData/raw/Ashland_Parcels.zip")
                parcels = geopandas.read_file(filePath)


In [ ]:
filePath = Path("../data/sampleData/raw/Ashland_Parcels.zip")
parcels = geopandas.read_file(filePath)

1. Search for owners (often under "deededOwner") and addresses (often split into columns by Census TIGER standards)
 and put them in the "owner" or "address" column.

By...

Viewing the columns

In [ ]:
parcels.head()

And then...

    1a. Either renaming...

            Example:
                    oldName = "DeededOwnr"
                    newName = "owner"
                    parcels = parcels.rename(columns={oldName:newName})

            or
                    oldName = "locAddress"
                    newName = "address"
                    parcels = parcels.rename(columns={oldName:newName})

In [ ]:
oldName = "DeededOwnr"
newName = "owner"

parcels = parcels.rename(columns={oldName:newName})

    1b. Or creating a new column (owner or address) and combining data.

        Example:
                newColumn = "address"
                column1 = "address"
                column2 = "address"
                column3 = ""


            parcels[newColumn] = parcels[column1].astype(str) + " " + parcels[column2].astype(str)

In [ ]:
newColumn = "address"

columns = ["mlocStrNo", "mlocStrNam"]


try:
    parcels[newColumn] = ""
    for clmn in columns:
                parcels[newColumn] = parcels[newColumn] + parcels[clmn].astype(str) + " "

    parcels[newColumn] = parcels[newColumn].str.strip()
except Exception as e:
    print("Error renaming columns")
    print(e)


    1c. The information might not be readily available without contacting the county auditor.

2. Then calculate its slope, acreage, aspect, and distance to grid.

    2a. To process a folder or file (parallel processing if more than 1 file), convert the folder to a list of file paths
     using the filesList tool, which will generate a list.  This will take approximately 5 minutes per file divided by
     the number of processor cores used.

            Example:
                    fileordirectory = "../data/sampleData/formatted/Ashland_Parcels.gpkg"
                    upload = False
                    cores = 1
                    parcelList = convert.filesList(fileordirectory)
                    slopeAspect.samePool(parcelList=parcelList, togrid=True, upload=False, save=Path("../data/parcels"))

In [ ]:
fileordirectory = "../data/sampleData/formatted/Ashland_Parcels.gpkg"
upload = True
# the number of cores used
cores = 1


parcelList = convert.filesList(fileordirectory)
slopeAspect.samePool(parcelList=parcelList, togrid=True, upload=upload, save=Path("../data/parcels", cores=cores))

3. If you chose not to upload the parcels to the database while processing, the files can be uploaded one at a time or as a batch.

    3a. To upload a file or folder by choosing postGIS as your chosen conversion method.  You will be prompted for your
     username and password.

In [ ]:
FileOrFolderToUpload = "../data/parcels/ashland_parcels.gpkg"

convert.convertToGenericGeoGEN("postgis", fileordirectory=FileOrFolderToUpload)

4. If the need arises, the files can also be converted to geojson, shapefile, and geopackage file formats.  Geometry errors
can sometimes be fixed this way.

In [ ]:
FileOrFolderToUpload = "../data/parcels/ashland_parcels.gpkg"

convert.convertToGenericGeoGEN("", fileordirectory=FileOrFolderToUpload)